## Eigendecomposition, Entropy Analysis, and Roe Averaging of the 1D Euler Equations for a Calorically Perfect Gas

A Demonstration of the Poblano Library

Mike Hansen

May, 2019

### Eigendecomposition of the Flux Jacobian Matrix

In [1]:
from poblano import PoblanoRing as Poblano
%display latex

First we build the `Poblano` object (named 'demo') with density, velocity, and temperature as primitive variables.

In [2]:
r = Poblano('demo', ['rho', 'u', 'T'])
rho, u, T = r.vars

v = vector([rho, u, T])

Next we build some derived quantities such as pressure and the isochoric specific heat capacity.

With the energy we are able to build the conserved state vector, `q`, and derive its Jacobian matrix with respect to the primitive variables.

In [3]:
var('R gamma')

p = rho * R * T
cv = R / (gamma - 1)
e = cv * T
E = e + u * u / 2

q = vector([rho, rho * u, rho * E])

qv = matrix(SR, 3, 3)
for i in range(3):
    for j in range(3):
        qv[i, j] = r.diff(q[i], v[j])

vq = qv.inverse().simplify_full()

show(qv)
show(vq)

[                        1                         0                         0]
[                        u                       rho                         0]
[1/2*u^2 + R*T/(gamma - 1)                     rho*u         R*rho/(gamma - 1)]

[                                    1                                     0                                     0]
[                               -u/rho                                 1/rho                                     0]
[1/2*((gamma - 1)*u^2 - 2*R*T)/(R*rho)                -(gamma - 1)*u/(R*rho)                   (gamma - 1)/(R*rho)]

Next we build the flux vector and compute its Jacobian with respect to primitives.

In [4]:
f = vector([rho * u, rho * u * u + p, (rho * E + p) * u])

fv = matrix(SR, 3, 3)
for i in range(3):
    for j in range(3):
        fv[i, j] = r.diff(f[i], v[j])

show(fv)

[                                                    u                                                   rho                                                     0]
[                                            R*T + u^2                                               2*rho*u                                                 R*rho]
[              1/2*(2*R*T + u^2 + 2*R*T/(gamma - 1))*u R*T*rho + rho*u^2 + 1/2*(u^2 + 2*R*T/(gamma - 1))*rho                         (R*rho + R*rho/(gamma - 1))*u]

Now we do a spectral decomposition of the flux Jacobian with respect to conserved variables ($f_q$) to obtain the wave speeds (and the speed of sound).

We compute the eigenvalues by decomposing a matrix that is _similar_ to $f_q$, $\mathcal{A}_v = v_q f_q q_v = v_q f_v$, because the algebra is far simpler.

In [5]:
av = (vq * fv).simplify_full()
show(LatexExpr('\mathcal{A}_v ='), av)

lam, sv = av.right_eigenmatrix()

show(LatexExpr('\Lambda ='), lam)

sos = lam[1, 1] - u
show('speed of sound = ', sos)

\mathcal{A}_v = [          u         rho           0]
[    R*T/rho           u           R]
[          0 T*gamma - T           u]

\Lambda = [u - sqrt(R*T*gamma)                   0                   0]
[                  0 u + sqrt(R*T*gamma)                   0]
[                  0                   0                   u]

'speed of sound = ' sqrt(R*T*gamma)

Next we derive eigenvectors of $f_q$, which are useful for boundary conditions, stencil biasing, and dissipative fluxes. We compute eigenvectors of $\mathcal{A}_v$ and use the similarity transform to obtain eigenvectors of $f_q$.

We make a `PoblanoExpression` to hide the definition of `a`, the speed of sound.

In [6]:
a = r.fxn('a', sos)

lam = matrix(SR, lam.ncols(), lam.nrows(), lam.dict(), sparse=False)
lam = lam.subs(R * T * gamma == a ** 2).canonicalize_radical().simplify_full()

sv = sv.subs(R * T * gamma == a ** 2).canonicalize_radical().simplify_full()
svi = sv.inverse().simplify_full()

sq = (qv * sv).simplify_full()
sqi = (svi * vq).simplify_full()

show(LatexExpr('\mathcal{A}_v ='), av, '=', sv, lam, svi)

show(LatexExpr('f_q ='), sq, lam, sqi)

\mathcal{A}_v = [          u         rho           0]
[    R*T/rho           u           R]
[          0 T*gamma - T           u] '=' [                1                 1                 1]
[           -a/rho             a/rho                 0]
[(T*gamma - T)/rho (T*gamma - T)/rho            -T/rho] [-a + u      0      0]
[     0  a + u      0]
[     0      0      u] [        1/2/gamma        -1/2*rho/a 1/2*rho/(T*gamma)]
[        1/2/gamma         1/2*rho/a 1/2*rho/(T*gamma)]
[(gamma - 1)/gamma                 0    -rho/(T*gamma)]

f_q = [                                                                  1                                                                   1                                                                   1]
[                                                             -a + u                                                               a + u                                                                   u]
[1/2*(2*R*T*gamma + (gamma - 1)*u^2 - 2*(a*gamma - a)*u)/(gamma - 1) 1/2*(2*R*T*gamma + (gamma - 1)*u^2 + 2*(a*gamma - a)*u)/(gamma - 1)                                                             1/2*u^2] [-a + u      0      0]
[     0  a + u      0]
[     0      0      u] [ 1/4*(2*R*T*gamma*u + (a*gamma - a)*u^2)/(R*T*a*gamma)       -1/2*(R*T*gamma + (a*gamma - a)*u)/(R*T*a*gamma)                            1/2*(gamma - 1)/(R*T*gamma)]
[-1/4*(2*R*T*gamma*u - (a*gamma - a)*u^2)/(R*T*a*gamma)        1/2*(R*T*gamma - (a*gamma - a)*u)/(R*T*a*gamma)                            1/2*(gamma - 1)/(R*T*gamma)]
[       1/2*(2*R*T*gamma - (gamma - 1)*u^2)/(R*T*gamma)                              (gamma - 1)*u/(R*T*gamma)                               -(gamma - 1)/(R*T*gamma)]

### Derivation of a Mathematical Entropy Function

The basis of entropy stability analysis is the entropy function and flux pair, $(\mathcal{S}, \mathcal{F})$.
Many admissible forms exist for the 1D Euler equations.
We choose the one based on the thermodynamic entropy, $s$, namely $\mathcal{S} = -\rho s$.
The corresponding entropy flux is $\mathcal{F} = u\mathcal{S}$.

In [7]:
s = R / (gamma - 1) * log(T) - R * log(rho)
S = r.fxn('S', -rho * s, latex_name='\\mathcal{S}')
F = r.fxn('F', S * u, latex_name='\\mathcal{F}')

show(S.dict)
show(F.dict)

{S: (R*log(rho) - R*log(T)/(gamma - 1))*rho}

{F: S*u}

Now we derive the entropy variables, $w=\mathcal{S}_q^\mathsf{T} = (\mathcal{S}_v v_q)^\mathsf{T}$.

Based on the entropy fluxes, functions, and variables, we compute the pair of the entropy potential $\phi=w^\mathsf{T}q - \mathcal{S}$ and entropy potential flux $\psi=w^\mathsf{T}f - \mathcal{F}$.

In [8]:
w = (vector([r.diff(S, rho), r.diff(S, u), r.diff(S, T)]) * vq).simplify()

show(LatexExpr('w ='), w)

phi = (w * q - S).subs(S.dict).simplify_full()
psi = (w * f - F).subs(F.dict).subs(S.dict).simplify_full()

show(LatexExpr('\phi ='), phi)
show(LatexExpr('\psi ='), psi)

w = (R*log(rho) + R - R*log(T)/(gamma - 1) - 1/2*((gamma - 1)*u^2 - 2*R*T)/(T*(gamma - 1)), u/T, -1/T)

\phi = R*rho

\psi = R*rho*u

To ensure that our entropy function is convex, we need to check that its Hessian matrix, $\mathcal{S}_{qq}=w_q=w_vv_q$, is symmetric and positive-definite.

Computing the matrix shows that it is indeed symmetric, although asking Sage to compute eigenvalues to show positive definiteness gives unwieldly results. Instead, we identify the diagonalization $\mathcal{D}=q_v^\mathsf{T}\mathcal{S}_{qq} q_v=q_v^\mathsf{T}w_q q_v=q_v^\mathsf{T}w_v$.

The existence of $\mathcal{S}_{qq}=ADA^\mathsf{T}$ with $A=v_q^\mathsf{T}$, combined with the symmetry of $\mathcal{S}_{qq}$ and Sylvester's law of inertia, shows that the positivity of $\mathcal{D}$ is required for positive definiteness of $\mathcal{S}_{qq}$.

As seen below, convexity of the entropy function are strict positivity of density and temperature.
Note that the entropy, due to $log(T)$ and $log(\rho)$, is not even well-defined if $\rho, T\leq0$.

In [9]:
wv = matrix(SR, 3, 3)
for i in range(3):
    for j in range(3):
        wv[i, j] = r.diff(w[i], v[j])

wq = (wv * vq).simplify_full()

show(LatexExpr('\mathcal{S}_{qq} ='), wq)
D = (qv.T * wv).simplify_full()
show(LatexExpr('\mathcal{D} ='), D)

show('Conditions for convexity of the entropy function:')
for i in range(3):
    show(D[i, i] > 0)

\mathcal{S}_{qq} = [1/4*(4*R^2*T^2*gamma + (gamma^2 - 2*gamma + 1)*u^4)/((R*T^2*gamma - R*T^2)*rho)                                                -1/2*(gamma - 1)*u^3/(R*T^2*rho)                                       1/2*((gamma - 1)*u^2 - 2*R*T)/(R*T^2*rho)]
[                                               -1/2*(gamma - 1)*u^3/(R*T^2*rho)                                             ((gamma - 1)*u^2 + R*T)/(R*T^2*rho)                                                      -(gamma - 1)*u/(R*T^2*rho)]
[                                      1/2*((gamma - 1)*u^2 - 2*R*T)/(R*T^2*rho)                                                      -(gamma - 1)*u/(R*T^2*rho)                                                         (gamma - 1)/(R*T^2*rho)]

\mathcal{D} = [                  R/rho                       0                       0]
[                      0                   rho/T                       0]
[                      0                       0 R*rho/(T^2*gamma - T^2)]

'Conditions for convexity of the entropy function:'

R/rho > 0

rho/T > 0

R*rho/(T^2*gamma - T^2) > 0

### Derivation of Entropy-Conservative Flux Functions

Above we proved that $\mathcal{S}$ is an entropy function for the one-dimensional Euler equations.

Now we derive two-point numerical flux functions which exactly conserve the entropy, by satisfying the local equality $(f^S)^\mathsf{T}\Delta w = \Delta \psi$, where $\Delta$ is the interface jump operator, $\Delta a = a_r - a_\ell$. Flux functions that satisfy this property are the basis of provably entropy stable numerical schemes.

We satisfy this identity by expanding the jumps of $w$ and $\psi$ with respect to a derivation vector of auxiliary $z$.
The resultant flux will depend heavily upon the choice of auxiliary variables $z$.
We first choose $z=v$ for convenience.

Given the expansions $\Delta w = H\Delta z$ and $\Delta \psi = g^\mathsf{T}\Delta z$, the local constraint for entropy conservation yields the flux $f^S = (H^\mathsf{T})^{-1}g$.

*We now utilize the highlight of Poblano: automatic jump expansion.*

In [10]:
show(w[0])

R*log(rho) + R - R*log(T)/(gamma - 1) - 1/2*((gamma - 1)*u^2 - 2*R*T)/(T*(gamma - 1))

In [11]:
g = vector(SR, 3)
H = matrix(SR, 3, 3)

for i in range(3):
    g[i] = r.jump_coeff(psi, v[i])
    for j in range(3):
        H[i, j] = r.jump_coeff(w[i], v[j])
        
show(LatexExpr('g ='), g)
show(LatexExpr('H ='), H)
fS = (H.T.inverse() * g).simplify()
show(LatexExpr('f^S ='), fS)

g = (R*demo_xi_9, R*demo_xi_3, 0)

H = [                                                                                   R/demo_xi_5                                                                          -demo_xi_12*demo_xi_9 R*demo_xi_12/(gamma - 1) - 1/2*demo_xi_11/(demo_xi_14*(gamma - 1)) - R/(demo_xi_7*(gamma - 1))]
[                                                                                             0                                                                                     demo_xi_12                                                                           demo_xi_9/demo_xi_14]
[                                                                                             0                                                                                              0                                                                                  -1/demo_xi_14]

f^S = (demo_xi_5*demo_xi_9, demo_xi_5*demo_xi_9^2 + R*demo_xi_3/demo_xi_12, 1/2*R*demo_xi_14*demo_xi_9*(2*demo_xi_5*demo_xi_9^2/(R*demo_xi_14) + demo_xi_5*(2*R*demo_xi_12/(gamma - 1) - demo_xi_11/(demo_xi_14*(gamma - 1)) - 2*R/(demo_xi_7*(gamma - 1)))/R) + R*demo_xi_3*demo_xi_9/demo_xi_12)

Now we make a very important check: we ensure that when the 'left' and 'right' states collapse the numerical flux function reduces to the physical flux - this is the _consistency_ of the numerical flux function.

Poblano makes it easy to check the behavior, with the `subs_for_consistency()` method.

In [12]:
for i in range(3):
    fSi_limit = r.subs_for_consistency(fS[i]).simplify_full()
    fphysi = f[i].simplify_full()
    show(LatexExpr('\lim_{q_\ell\\to q_r}f^S_' + str(i) + ' = ' + latex(fSi_limit) + '\\stackrel{?}{=}' + latex(fphysi) + ' = f_{\mathrm{phys},' + str(i) + '} \quad \\text{' + str(bool((fphysi - fSi_limit)==0)) + '}'))

\lim_{q_\ell\to q_r}f^S_0 = \rho u \stackrel{?}{=} \rho u = f_{\mathrm{phys}, 0 } \quad \text{ True }

\lim_{q_\ell\to q_r}f^S_1 = R T \rho + \rho u^{2} \stackrel{?}{=} R T \rho + \rho u^{2} = f_{\mathrm{phys}, 1 } \quad \text{ True }

\lim_{q_\ell\to q_r}f^S_2 = \frac{2 \, R T \gamma \rho u + {\left(\gamma - 1\right)} \rho u^{3}}{2 \, {\left(\gamma - 1\right)}} \stackrel{?}{=} \frac{2 \, R T \gamma \rho u + {\left(\gamma - 1\right)} \rho u^{3}}{2 \, {\left(\gamma - 1\right)}} = f_{\mathrm{phys}, 2 } \quad \text{ True }

Above we showed that an entropy-conservative numerical flux function can be derived with the primitive variables used to expand the jumps, and that the resultant flux is consistent.

Next we use auxiliary variables of $z=(\rho,u,Z)$ where $Z=1/T$, motivated by the prevalance of the inverse temperature in the flux function above (due to the entropy variables including $1/T$).

To derive a flux function with this modified vector, we build a new `Poblano` object with the new auxiliary variables, substitute $T=1/Z$ into the expressions we need, and then compute $f^S$ as above. We show that the flux function is consistent. Note that this flux is far simpler than the one derived with temperature (instead of the inverse temperature) in the parameter vector.

In [13]:
rz = Poblano('with_invT', [rho, u, 'Z'])
Z = rz.vars[2]
wz = w.subs(T = 1 / Z)
for i in range(3):
    wz[i] = wz[i].expand().log_expand()
psiz = psi.subs(T = 1 / Z).simplify_full()
show(wz)
show(psiz)

gz = vector(SR, 3)
Hz = matrix(SR, 3, 3)

for i in range(3):
    gz[i] = rz.jump_coeff(psiz, rz.vars[i])
    for j in range(3):
        Hz[i, j] = rz.jump_coeff(wz[i], rz.vars[j]).simplify_full()
fSz = (Hz.T.inverse() * gz).simplify()

show(LatexExpr('g ='), gz)
show(LatexExpr('H ='), Hz)
show(LatexExpr('f^S ='), fSz)

(-1/2*Z*gamma*u^2/(gamma - 1) + 1/2*Z*u^2/(gamma - 1) + R*log(rho) + R + R*log(Z)/(gamma - 1) + R/(gamma - 1), Z*u, -Z)

R*rho*u

g = (R*with_invT_xi_2, R*with_invT_xi_1, 0)

H = [                                                                   R/with_invT_xi_6                                                      -with_invT_xi_2*with_invT_xi_3 -1/2*((gamma - 1)*with_invT_xi_4*with_invT_xi_8 - 2*R)/((gamma - 1)*with_invT_xi_8)]
[                                                                                  0                                                                      with_invT_xi_3                                                                      with_invT_xi_2]
[                                                                                  0                                                                                   0                                                                                  -1]

f^S = (with_invT_xi_2*with_invT_xi_6, with_invT_xi_2^2*with_invT_xi_6 + R*with_invT_xi_1/with_invT_xi_3, 1/2*(2*with_invT_xi_2^2*with_invT_xi_6/R - ((gamma - 1)*with_invT_xi_4*with_invT_xi_8 - 2*R)*with_invT_xi_6/(R*(gamma - 1)*with_invT_xi_8))*R*with_invT_xi_2 + R*with_invT_xi_1*with_invT_xi_2/with_invT_xi_3)

In [14]:
for i in range(3):
    fSi_limit = rz.subs_for_consistency(fSz[i]).subs(Z=1/T).simplify_full()
    fphysi = f[i].simplify_full()
    show(LatexExpr('\lim_{q_\ell\\to q_r}f^S_' + str(i) + ' = ' + latex(fSi_limit) + '\\stackrel{?}{=}' + latex(fphysi) + ' = f_{\mathrm{phys},' + str(i) + '} \quad \\text{' + str(bool((fphysi - fSi_limit)==0)) + '}'))

\lim_{q_\ell\to q_r}f^S_0 = \rho u \stackrel{?}{=} \rho u = f_{\mathrm{phys}, 0 } \quad \text{ True }

\lim_{q_\ell\to q_r}f^S_1 = R T \rho + \rho u^{2} \stackrel{?}{=} R T \rho + \rho u^{2} = f_{\mathrm{phys}, 1 } \quad \text{ True }

\lim_{q_\ell\to q_r}f^S_2 = \frac{2 \, R T \gamma \rho u + {\left(\gamma - 1\right)} \rho u^{3}}{2 \, {\left(\gamma - 1\right)}} \stackrel{?}{=} \frac{2 \, R T \gamma \rho u + {\left(\gamma - 1\right)} \rho u^{3}}{2 \, {\left(\gamma - 1\right)}} = f_{\mathrm{phys}, 2 } \quad \text{ True }

### Determining the Roe Average Matrix

The final part of this demonstration for the 1D Euler equations is computation of the 'Roe average' matrix.
This matrix $\mathcal{A}$ comes from the idea of locally replacing the nonlinear Euler equations, $q_t + f_x = 0$, with a linear approximation, $q_t + \mathcal{A}q_x = 0$.

The matrix $\mathcal{A}$ is a two-point approximation of $f_q$ and satisfies, for _any_ $q_\ell$ and $q_r$, $\Delta f = \mathcal{A}\Delta q$. Furthermore it must be consistent, $\lim_{q_\ell \to q_r}\mathcal{A}=f_q$.

To obtain $\mathcal{A}$ we perform a jump expansion as above, $\mathcal{C}\Delta z = \mathcal{A}\mathcal{B}\Delta z$, where $\Delta f = \mathcal{C}\Delta z$ and $\Delta q = \mathcal{B}\Delta z$. This yields $\mathcal{A} = \mathcal{C}\mathcal{B}^{-1}$. With Poblano's jump expansion capabilities, we can compute this in straightforward fashion. First we show this result with $z=v$, and afterwards we show $z$ with inverse temperature, and then with the original parameter vector that Roe used. It's clear that Roe's parameter vector yields the cleanest expression for $\mathcal{A}$.

In [15]:
B = matrix(SR, 3, 3)
C = matrix(SR, 3, 3)

for i in range(3):
    for j in range(3):
        B[i, j] = r.jump_coeff(q[i], r.vars[j])
        C[i, j] = r.jump_coeff(f[i], r.vars[j])

A = (C * B.inverse()).simplify_full()
show(LatexExpr('\mathcal{B} ='), B)
show(LatexExpr('\mathcal{C} ='), C)
show(LatexExpr('\mathcal{A} ='), A)

\mathcal{B} = [                      1                       0                       0]
[              demo_xi_9               demo_xi_3                       0]
[         1/2*demo_xi_16     demo_xi_3*demo_xi_9 R*demo_xi_3/(gamma - 1)]

\mathcal{C} = [                                        demo_xi_9                                         demo_xi_3                                                 0]
[                         R*demo_xi_1 + demo_xi_13                             2*demo_xi_3*demo_xi_9                                       R*demo_xi_3]
[       1/2*(2*R*demo_xi_1 + demo_xi_16)*demo_xi_9            demo_xi_3*demo_xi_9^2 + 1/2*demo_xi_17 (R*demo_xi_3 + R*demo_xi_3/(gamma - 1))*demo_xi_9]

\mathcal{A} = [                                                                                                                                                                            0                                                                                                                                                                             1                                                                                                                                                                             0]
[                                                                           R*demo_xi_1 - 3*demo_xi_9^2 + 1/2*(2*demo_xi_9^2 - demo_xi_16)*gamma + demo_xi_13 + 1/2*demo_xi_16                                                                                                                                                -demo_xi_9*gamma + 3*demo_xi_9                                                                                                                                                                     gamma - 1]
[-1/2*(2*demo_xi_3*demo_xi_9^3 - ((2*R*demo_xi_1 + demo_xi_16)*demo_xi_3 - demo_xi_17)*demo_xi_9 - (2*demo_xi_3*demo_xi_9^3 - demo_xi_16*demo_xi_3*demo_xi_9)*gamma)/demo_xi_3                                                                                         -1/2*(2*demo_xi_3*demo_xi_9^2*gamma - 2*demo_xi_3*demo_xi_9^2 - demo_xi_17)/demo_xi_3                                                                                                                                                               demo_xi_9*gamma]

In [16]:
Bz = matrix(SR, 3, 3)
Cz = matrix(SR, 3, 3)

for i in range(3):
    for j in range(3):
        Bz[i, j] = rz.jump_coeff(q[i].subs(T=1/Z), rz.vars[j])
        Cz[i, j] = rz.jump_coeff(f[i].subs(T=1/Z), rz.vars[j])

Az = (Cz * Bz.inverse()).simplify_full()
show(LatexExpr('\mathcal{B} ='), Bz)
show(LatexExpr('\mathcal{C} ='), Cz)
show(LatexExpr('\mathcal{A} ='), Az)

\mathcal{B} = [                                             1                                              0                                              0]
[                                with_invT_xi_2                                 with_invT_xi_1                                              0]
[                           1/2*with_invT_xi_12                  with_invT_xi_1*with_invT_xi_2 R*with_invT_xi_1/((gamma - 1)*with_invT_xi_11)]

\mathcal{C} = [                                                                                    with_invT_xi_2                                                                                     with_invT_xi_1                                                                                                  0]
[                                                                R*with_invT_xi_10 + with_invT_xi_4                                                                    2*with_invT_xi_1*with_invT_xi_2                                                                   R*with_invT_xi_1/with_invT_xi_11]
[                                        1/2*(2*R*with_invT_xi_10 + with_invT_xi_12)*with_invT_xi_2                                              with_invT_xi_1*with_invT_xi_2^2 + 1/2*with_invT_xi_13 (R*with_invT_xi_1/with_invT_xi_11 + R*with_invT_xi_1/((gamma - 1)*with_invT_xi_11))*with_invT_xi_2]

\mathcal{A} = [                                                                                                                                                                                      0                                                                                                                                                                                       1                                                                                                                                                                                       0]
[                                                                                    (gamma - 3)*with_invT_xi_2^2 + R*with_invT_xi_10 - 1/2*(gamma - 1)*with_invT_xi_12 + with_invT_xi_4                                                                                                                                                             -(gamma - 3)*with_invT_xi_2                                                                                                                                                                               gamma - 1]
[1/2*(2*(gamma - 1)*with_invT_xi_1*with_invT_xi_2^3 + (2*R*with_invT_xi_1*with_invT_xi_10 - (gamma - 1)*with_invT_xi_1*with_invT_xi_12 - with_invT_xi_13)*with_invT_xi_2)/with_invT_xi_1                                                                                                   -1/2*(2*(gamma - 1)*with_invT_xi_1*with_invT_xi_2^2 - with_invT_xi_13)/with_invT_xi_1                                                                                                                                                                    gamma*with_invT_xi_2]

In [17]:
rroe = Poblano('Roe', ['z_1', 'z_2', 'z_3'], latex_name_dict={'z_1': '\\mathcal{W}_1', 'z_2': '\\mathcal{W}_2', 'z_3': '\\mathcal{W}_3'})
Z1, Z2, Z3 = rroe.vars

to_roe_dict = {rho: Z1 ** 2, u: Z2 / Z1, T: (Z3 / Z1 - Z2 ** 2 / Z1 ** 2 / 2) / (cv + R)}

Br = matrix(SR, 3, 3)
Cr = matrix(SR, 3, 3)

for i in range(3):
    for j in range(3):
        Br[i, j] = rroe.jump_coeff(q[i].subs(to_roe_dict).simplify_full().expand(), rroe.vars[j])
        Cr[i, j] = rroe.jump_coeff(f[i].subs(to_roe_dict).simplify_full().expand(), rroe.vars[j])

Br = Br.simplify_full()
Cr = Cr.simplify_full()
Ar = (Cr * Br.inverse()).simplify_full()
show(LatexExpr('\mathcal{B} ='), Br)
show(LatexExpr('\mathcal{C} ='), Cr)
show(LatexExpr('\mathcal{A} ='), Ar)

\mathcal{B} = [                       2*Roe_xi_0                                 0                                 0]
[                         Roe_xi_1                          Roe_xi_0                                 0]
[                   Roe_xi_3/gamma (Roe_xi_1*gamma - Roe_xi_1)/gamma                    Roe_xi_0/gamma]

\mathcal{C} = [                         Roe_xi_1                          Roe_xi_0                                 0]
[(Roe_xi_3*gamma - Roe_xi_3)/gamma (Roe_xi_1*gamma + Roe_xi_1)/gamma (Roe_xi_0*gamma - Roe_xi_0)/gamma]
[                                0                          Roe_xi_3                          Roe_xi_1]

\mathcal{A} = [                                                                            0                                                                             1                                                                             0]
[                             1/2*(Roe_xi_1^2*gamma - 3*Roe_xi_1^2)/Roe_xi_0^2                                       -(Roe_xi_1*gamma - 3*Roe_xi_1)/Roe_xi_0                                                                     gamma - 1]
[1/2*(Roe_xi_1^3*gamma - Roe_xi_1^3 - 2*Roe_xi_0*Roe_xi_1*Roe_xi_3)/Roe_xi_0^3               -(Roe_xi_1^2*gamma - Roe_xi_1^2 - Roe_xi_0*Roe_xi_3)/Roe_xi_0^2                                                       Roe_xi_1*gamma/Roe_xi_0]

#### Using the Roe parameter vector to derive an entropy conservative flux function

In this little addendum, we show that while Roe's parameter vector is terrific for the Roe average matrix, it is useless for deriving entropy conservative flux functions, because the entropy variables are so convoluted when written in terms of the parameter vector. We don't try to solve and typeset the flux function as it will crash one's computer...

In [20]:
wroe = w.subs(to_roe_dict)
for i in range(3):
    wroe[i] = wroe[i].canonicalize_radical().simplify_full()
psiroe = psi.subs(to_roe_dict).simplify_full()
show(wroe)
show(psiroe)

(((2*R*gamma*log(z_1) + 2*R*gamma + R*log(2) + R*log(R) - R*log(gamma - 1) + R*log(gamma))*z_2^2 - 2*(2*R*gamma*z_1*log(z_1) + (R*gamma + R*log(2) + R*log(R) - R*log(gamma - 1) + R*log(gamma))*z_1)*z_3 - (R*z_2^2 - 2*R*z_1*z_3)*log(-z_2^2 + 2*z_1*z_3))/((gamma - 1)*z_2^2 - 2*(gamma - 1)*z_1*z_3), -2*R*gamma*z_1*z_2/((gamma - 1)*z_2^2 - 2*(gamma - 1)*z_1*z_3), 2*R*gamma*z_1^2/((gamma - 1)*z_2^2 - 2*(gamma - 1)*z_1*z_3))

R*z_1*z_2

Roe _is_ responsible for an entropy conservative flux function, derived with a different parameter vector, defined as $\theta_1=\sqrt{\rho/p}$, $\theta_2=\sqrt{\rho/p}u$, $\theta_3=\sqrt{\rho/p}p$. We show this derivation here, which we note is very straightforward and fast with Poblano (compared to doing the algebra entirely manually). This will be the third entropy conservative flux function we've derived, all without tedious, error-prone math by hand.

In [28]:
rroe2 = Poblano('Roe2', ['z_12', 'z_22', 'z_32'], latex_name_dict={'z_12': '\\theta_1', 'z_22': '\\theta_2', 'z_32': '\\theta_3'})
T1, T2, T3 = rroe2.vars

to_roe2_dict = {rho: T1 * T3, u: T2 / T1, T: 1 / (R * T1 ** 2)}

wroe2 = w.subs(to_roe2_dict)
for i in range(3):
    wroe2[i] = wroe2[i].canonicalize_radical().simplify_full().expand()
psiroe2 = psi.subs(to_roe2_dict).simplify_full()
show(wroe2)
show(psiroe2)

groe2 = vector(SR, 3)
Hroe2 = matrix(SR, 3, 3)

for i in range(3):
    groe2[i] = rroe2.jump_coeff(psiroe2, rroe2.vars[i])
    for j in range(3):
        Hroe2[i, j] = rroe2.jump_coeff(wroe2[i], rroe2.vars[j]).simplify_full()
fSroe2 = (Hroe2.T.inverse() * groe2).simplify_full()

show(LatexExpr('g ='), groe2)
show(LatexExpr('H ='), Hroe2)
show(LatexExpr('f^S ='), fSroe2.simplify_full())

(-1/2*R*gamma*z_22^2/(gamma - 1) + 1/2*R*z_22^2/(gamma - 1) + R*gamma*log(z_12)/(gamma - 1) + R*gamma*log(z_32)/(gamma - 1) + R*gamma/(gamma - 1) + R*log(R)/(gamma - 1) + R*log(z_12)/(gamma - 1) - R*log(z_32)/(gamma - 1), R*z_12*z_22, -R*z_12^2)

R*z_22*z_32

g = (0, R*Roe2_xi_2, R*Roe2_xi_1)

H = [(R*gamma + R)/(Roe2_xi_5*gamma - Roe2_xi_5)                                -R*Roe2_xi_1                                 R/Roe2_xi_7]
[                                R*Roe2_xi_1                                 R*Roe2_xi_9                                           0]
[                             -2*R*Roe2_xi_9                                           0                                           0]

f^S = (Roe2_xi_1*Roe2_xi_7, (Roe2_xi_1^2*Roe2_xi_7 + Roe2_xi_2)/Roe2_xi_9, -1/2*(Roe2_xi_1^3*Roe2_xi_5*Roe2_xi_7 + Roe2_xi_1*Roe2_xi_2*Roe2_xi_5 - Roe2_xi_1*Roe2_xi_7*Roe2_xi_9 - (Roe2_xi_1^3*Roe2_xi_5*Roe2_xi_7 + Roe2_xi_1*Roe2_xi_2*Roe2_xi_5 + Roe2_xi_1*Roe2_xi_7*Roe2_xi_9)*gamma)/(Roe2_xi_5*Roe2_xi_9^2*gamma - Roe2_xi_5*Roe2_xi_9^2))

#### Using the Chandrashekar parameter vector to derive an entropy conservative flux function

Just for fun... we'll derive yet another entropy conservative flux function, this one with the parameter vector of Chandrashekar, $\pi_1=\rho$, $\pi_2=u$, and $\pi_3=1/(2RT)$, only slightly different than our second parameter vector that simply uses the inverse temperature. Comparison shows that the resultant flux functions are pretty similar (Sage does some different simplications, but oh well).

In [38]:
rckr = Poblano('Roe2', [rho, u, 'Z_ckr'], latex_name_dict={'Z_ckr': '\\pi_3'})
Z_ckr = rckr.vars[-1]

to_ckr_dict = {T: 1 / (2 * R * Z_ckr)}

wckr = w.subs(to_ckr_dict)
for i in range(3):
    wckr[i] = wckr[i].canonicalize_radical().simplify_full().expand()
psickr = psi.subs(to_ckr_dict).simplify_full()
show(wckr)
show(psickr)

gckr = vector(SR, 3)
Hckr = matrix(SR, 3, 3)

for i in range(3):
    gckr[i] = rckr.jump_coeff(psickr, rckr.vars[i])
    for j in range(3):
        Hckr[i, j] = rckr.jump_coeff(wckr[i], rckr.vars[j]).simplify_full().expand()
fSckr = (Hckr.T.inverse() * gckr).simplify()

show(LatexExpr('g ='), gckr)
show(LatexExpr('H ='), Hckr)
show(LatexExpr('\\text{Chandrashekar: } f^S ='), fSckr)
show(LatexExpr('\\text{simple 1/T: } f^S ='), fSz)

(-R*Z_ckr*gamma*u^2/(gamma - 1) + R*Z_ckr*u^2/(gamma - 1) + R*gamma*log(rho)/(gamma - 1) + R*gamma/(gamma - 1) + R*log(2)/(gamma - 1) + R*log(R)/(gamma - 1) + R*log(Z_ckr)/(gamma - 1) - R*log(rho)/(gamma - 1), 2*R*Z_ckr*u, -2*R*Z_ckr)

R*rho*u

g = (R*Roe2_xi_2, R*Roe2_xi_1, 0)

H = [                                                                                                                                       R/Roe2_xi_7                                                                                                                           -2*R*Roe2_xi_2*Roe2_xi_4 -R*Roe2_xi_5*Roe2_xi_9*gamma/(Roe2_xi_9*gamma - Roe2_xi_9) + R*Roe2_xi_5*Roe2_xi_9/(Roe2_xi_9*gamma - Roe2_xi_9) + R/(Roe2_xi_9*gamma - Roe2_xi_9)]
[                                                                                                                                                 0                                                                                                                                      2*R*Roe2_xi_4                                                                                                                                      2*R*Roe2_xi_2]
[                                                                                                                                                 0                                                                                                                                                  0                                                                                                                                               -2*R]

\text{Chandrashekar: } f^S = (Roe2_xi_2*Roe2_xi_7, Roe2_xi_2^2*Roe2_xi_7 + 1/2*Roe2_xi_1/Roe2_xi_4, 1/2*(2*Roe2_xi_2^2*Roe2_xi_7 - (R*Roe2_xi_5*Roe2_xi_9*gamma/(Roe2_xi_9*gamma - Roe2_xi_9) - R*Roe2_xi_5*Roe2_xi_9/(Roe2_xi_9*gamma - Roe2_xi_9) - R/(Roe2_xi_9*gamma - Roe2_xi_9))*Roe2_xi_7/R)*Roe2_xi_2 + 1/2*Roe2_xi_1*Roe2_xi_2/Roe2_xi_4)

\text{simple 1/T: } f^S = (with_invT_xi_2*with_invT_xi_6, with_invT_xi_2^2*with_invT_xi_6 + R*with_invT_xi_1/with_invT_xi_3, 1/2*(2*with_invT_xi_2^2*with_invT_xi_6/R - ((gamma - 1)*with_invT_xi_4*with_invT_xi_8 - 2*R)*with_invT_xi_6/(R*(gamma - 1)*with_invT_xi_8))*R*with_invT_xi_2 + R*with_invT_xi_1*with_invT_xi_2/with_invT_xi_3)